<a href="https://colab.research.google.com/github/ananyanivarthi/profile_parsingg/blob/main/AImocktestapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ** Profile Parsing Model**

In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.1 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 54.8 MB/s eta 0:00:00


In [3]:
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 37.8 MB/s eta 0:00:00
✔ Downlo

In [6]:
def entities():
    import requests
    import spacy
    import nltk
    import string
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize, sent_tokenize
    import pandas as pd
    import json

    nltk.download('stopwords')
    nltk.download('punkt')

    nlp = spacy.load("en_core_web_trf")

    skills_list = [
        "Python", "Machine Learning", "Deep Learning", "Data Science", "Artificial Intelligence", "SQL", "NoSQL", "Java",
        "C++", "HTML", "CSS", "JavaScript", "TensorFlow", "PyTorch", "R", "Pandas", "AWS", "Azure", "AutoCAD", "SolidWorks",
        "SPICE", "MS Project", "Lean Manufacturing", "Excel", "QuickBooks", "SEO", "PPC", "Agile", "Scrum", "Tableau",
        "Google Analytics", "MERN", "React", "Node", "Express", "MongoDB", "regularization", "l1", "l2"
    ]

    skills_set = set(skill.lower() for skill in skills_list)

    def fetch_component_data(url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Failed to fetch data from {url}: {e}")
            return None

    def preprocess_text(text):
        text = text.lower()
        text = text.translate(str.maketrans("", "", string.punctuation))
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in word_tokenize(text) if word not in stop_words]
        lemmatized_words = [nlp(word)[0].lemma_ for word in filtered_words]
        return " ".join(lemmatized_words)

    def evaluate_skill_level(skill, work_experience_data, projects_data, education_data):
        points = 0

        if work_experience_data:
            for experience in work_experience_data[0]['work_experience']:
                if skill.lower() in experience['job_responsibilities'].lower():
                    start_date = experience['start_date']
                    end_date = experience['end_date']
                    years_of_experience = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365
                    points += years_of_experience * 2

        if projects_data:
            for project in projects_data:
                if skill.lower() in project['technology_used'].lower():
                    points += 3

        if education_data:
            for education in education_data['education_records']:
                if skill.lower() in education['field_of_study'].lower():
                    points += 5

        if points >= 10:
            return "Expert"
        elif 5 <= points < 10:
            return "Intermediate"
        else:
            return "Beginner"

    email = input("Please enter the user's email address: ").strip()

    projects_url = f"https://aitq-api-t4yr74gyka-uc.a.run.app/projects/get?email={email}"
    work_experience_url = f"https://test-t4yr74gyka-uc.a.run.app/work_experience/get?email={email}"
    education_url = f"https://test-t4yr74gyka-uc.a.run.app/education/get?email={email}"
    skills_url = f"https://test-862291870424.us-central1.run.app/skills/get?email={email}"

    projects_data = fetch_component_data(projects_url)
    work_experience_data = fetch_component_data(work_experience_url)
    education_data = fetch_component_data(education_url)
    skills_data = fetch_component_data(skills_url)

    extracted_info = ""

    if projects_data:
        extracted_info += "Projects:\n"
        extracted_info += "\n".join([f"Title: {project['project_title']} \nDescription: {project['description']} \nTechnology: {project['technology_used']} \nLink: {project['project_link']}\n" for project in projects_data])

    if work_experience_data:
        extracted_info += "\nWork Experience:\n"
        for experience in work_experience_data[0]['work_experience']:
            extracted_info += f"\nCompany: {experience['company_name']} \nJob Title: {experience['job_title']} \nEmployment Type: {experience['employment_type']} \nResponsibilities: {experience['job_responsibilities']} \nStart Date: {experience['start_date']} \nEnd Date: {experience['end_date']}\n"

    if education_data:
        extracted_info += "\nEducation:\n"
        for education in education_data['education_records']:
            extracted_info += f"\nDegree: {education['highest_qualification']} \nField of Study: {education['field_of_study']} \nInstitution: {education['institution_name']} \nLocation: {education['city_state']}, {education['country']} \nStart Date: {education['start_date']} \nEnd Date: {education['end_date']}\n"

    if skills_data:
        extracted_info += "\nSkills:\n"
        for skill in skills_data:
            extracted_info += f"\nSkill: {skill['skill']} \nYears of Experience: {skill['yr_of_experience']}\n"

    if not extracted_info:
        print("No data available or failed to fetch data.")
        return

    preprocessed_text = preprocess_text(extracted_info)

    words = preprocessed_text.split()

    # Use a dictionary to store the latest skill and its level
    skill_dict = {}

    for word in words:
        if word in skills_set:
            skill_level = evaluate_skill_level(word, work_experience_data, projects_data, education_data)
            skill_dict[word] = skill_level  # This will overwrite duplicate skills with the latest level

    # Save the results in a JSON file
    entities_file_path = "extracted_entities.json"
    try:
        with open(entities_file_path, 'w') as json_file:
            json.dump(skill_dict, json_file, indent=4)
        print(f"Skill levels saved to '{entities_file_path}'.")
    except IOError as e:
        print(f"Failed to write entities to {entities_file_path}: {e}")

    # Output the results
    for entity, skill_level in skill_dict.items():
        print(f"{entity}: {skill_level}")

entities()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Please enter the user's email address: ananyanivarthi@gmail.com
Skill levels saved to 'extracted_entities.json'.
regularization: Intermediate
l1: Intermediate
l2: Intermediate
mern: Beginner
mongodb: Beginner
express: Beginner
react: Beginner
python: Beginner
java: Beginner
